In [1]:
# import lib
import sys
import os
import pandas as pd
import argparse
from yolo import YOLO, detect_video
from PIL import Image

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
args = {'model': YOLO.get_defaults("model_path"),
        'anchors': YOLO.get_defaults("anchors_path"),
        'classes': YOLO.get_defaults("classes_path"),
        'gpu_num': YOLO.get_defaults("gpu_num"),
        'input': 'img_sample',
        'output': 'output',
        'score_file': 'submission.csv',
        'class_file': 'model_data/oi_classes.csv',
        'noimgdet': False,
       }

In [3]:
yolo = YOLO(**args)
os.makedirs(yolo.output, exist_ok=True)

model_data/yolo.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
# load Open Images classes
oi_class = pd.read_csv(args['class_file'])
display(oi_class)

,Label,Name
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football
...,...,...
596,/m/0qmmr,Wheelchair
597,/m/0wdt60w,Rugby ball
598,/m/0xfy,Armadillo
599,/m/0xzly,Maracas


In [17]:
df = pd.DataFrame(columns=['ImageID', 'PredictionString'])
for img_name in os.listdir(yolo.input):
    try:
        image = Image.open(os.path.join(yolo.input, img_name))
    except:
        print('Open Error!')
        print(img_name)
        continue
    else:
        out_boxes, out_scores, out_classes, r_image = yolo.detect_image(image)

    # save image
    r_image.save(os.path.join(yolo.output, img_name))

    pred_str = ""
    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = yolo.class_names[c]
        box = out_boxes[i]
        score = out_scores[i]
        
        # search class label
        mask = oi_class['Name'] == str.capitalize(predicted_class)
        labels = oi_class[mask]['Label'].values
        
        # labels is empty, class not found
        if(labels.size == 0):
            continue
            
        label = labels[0]
        top, left, bottom, right = box
        xmin = left / image.width
        xmax = right / image.width
        ymin = top / image.height
        ymax = bottom / image.height
        pred_str += " {} {} {} {} {} {}".format(label, score, xmin, ymin, xmax, ymax)
        print(f"label:{label}|name:{predicted_class}|score:{score}|xmin:{xmin}|ymin:{ymin}|xmax:{xmax}|ymax:{ymax}")
        # print(f"r_image.width:{r_image.width} | r_image.height:{r_image.height}")
        # print(f"image.width:{image.width} | image.height:{image.height}")

    # save score
    imageid = os.path.splitext(os.path.basename(img_name))[0]
    s = pd.Series([imageid, pred_str], index=df.columns)
    df = df.append(s, ignore_index=True)

df.to_csv(args['score_file'], index=False, header=True)

handbag 0.49 (624, 441) (698, 545)
truck 0.36 (780, 159) (1011, 382)
truck 0.45 (898, 149) (1015, 413)
truck 0.99 (64, 80) (622, 584)
person 0.35 (615, 288) (698, 412)
person 0.91 (583, 289) (655, 410)
person 0.94 (759, 250) (876, 605)
person 0.98 (883, 210) (976, 571)
person 0.99 (454, 274) (549, 597)
--------------------
elapsed time: 1.2747483379989717s
--------------------
label:/m/080hkjn|name:handbag|score:0.4898366928100586|xmin:0.6091075539588928|ymin:0.7982973068036238|xmax:0.6816160082817078|ymax:0.6454314068505216
label:/m/07r04|name:truck|score:0.3620006740093231|xmin:0.7612306475639343|ymin:0.559659139652727|xmax:0.9877368807792664|ymax:0.23336539540772558
label:/m/07r04|name:truck|score:0.4495103657245636|xmin:0.8767225742340088|ymin:0.6039731652635432|xmax:0.9908020496368408|ymax:0.21755583778481882
label:/m/07r04|name:truck|score:0.9935256242752075|xmin:0.06283590197563171|ymin:0.8545866445426885|xmax:0.6076253652572632|ymax:0.11710772088471129
label:/m/01g317|name:pers

In [6]:
display(df)

,ImageID,PredictionString
0,00000b4dcff7f799,/m/080hkjn 0.4898366928100586 0.6091075539588...
1,00001a21632de752,/m/0bt_c3 0.559672474861145 0.489931493997573...
2,0000d67245642c5f,


In [7]:
#yolo.close_session()